# Customer Segmentation: A Systematic Approach from Raw Data to Actionable Insights

### 1. Project Objective & Methodology

The goal of this project is to perform an end-to-end customer segmentation analysis on the raw Online Retail dataset. We will systematically process and transform the raw transactional data into a rich, multi-dimensional customer profile. This profile will then be used with unsupervised machine learning to identify distinct, actionable customer segments.

Our methodology emphasizes a rigorous and reproducible workflow:

1.  **Environment Setup:** Import all necessary libraries and configure the notebook environment for consistency.
2.  **Data Ingestion:** Load the raw dataset from its source.
3.  **Data Cleaning & Preprocessing:** Rigorously clean the raw data to handle missing values, duplicates, and invalid entries, creating a reliable analytical base.
4.  **Advanced Feature Engineering:** Move beyond traditional RFM by creating a custom 8-feature **"Customer DNA"** profile that captures profitability, loyalty, and purchasing behavior.
5.  **Exploratory Data Analysis (EDA):** Thoroughly analyze and visualize the engineered features to understand their statistical properties and relationships.
6.  **Modeling & Segmentation:** Apply and evaluate clustering algorithms to segment the customer base.

This notebook will serve as a definitive guide, with each step clearly documented to ensure clarity and reproducibility.

In [ ]:
# =============================================================================
# Block 1: Environment Setup
# =============================================================================

# --- 1.1 Core Libraries for Data Manipulation ---
import pandas as pd
import numpy as np
import datetime as dt

# --- 1.2 Libraries for Data Visualization ---
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# --- 1.3 Libraries for Machine Learning ---
# Preprocessing & Dimensionality Reduction
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Modeling (to be used in the modeling phase)
from sklearn.cluster import KMeans, DBSCAN

# --- 1.4 Configure Notebook Environment ---
# Set a consistent, professional style for all plots
sns.set_style("whitegrid")
# Suppress scientific notation for floats for better readability
pd.options.display.float_format = '{:,.2f}'.format
# Set default figure size
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ All libraries imported and environment configured successfully.")

### 2. Data Ingestion

In this step, we will acquire the raw dataset. We are working exclusively from the `OnlineRetail.xlsx` file, ensuring our entire analysis is built from the ground up from a single source of truth.

* **What it does:** It uses the `gdown` library to download the raw data file directly from its shared Google Drive link.
* **Why we do this:** This automates the data acquisition process, making the notebook self-contained and easily reproducible by any team member or professor. After downloading, it loads the data into a pandas DataFrame for initial inspection.

In [ ]:
# =============================================================================
# Block 2: Data Ingestion - Downloading & Loading the Raw File
# =============================================================================

# --- 2.1 Install gdown for Google Drive Downloads ---
# We run this quietly as it only needs to happen once per session
!pip install gdown --quiet

# --- 2.2 Download the Raw Dataset ---
# The file ID for the 'OnlineRetail.xlsx' Google Sheet
file_id = '17hbtFKg70j5x_QN8F0dBZmsHxwkFKIGawmQ9_uAfFU0'
output_filename = 'OnlineRetail.xlsx'

print(f"⬇️ Downloading raw data file: '{output_filename}'...")
try:
    # gdown automatically detects a Google Sheet and downloads it as an .xlsx file
    !gdown $file_id -O $output_filename
    print(f"✅ File downloaded successfully as '{output_filename}'.")
except Exception as e:
    print(f"❌ ERROR: File download failed. Please check the file ID and sharing permissions.")
    print(e)


# --- 2.3 Load the Dataset into a DataFrame ---
try:
    df_raw = pd.read_excel(output_filename)
    print(f"\n✅ Raw dataset loaded successfully.")
    print(f"Shape of the dataset: {df_raw.shape}")
except FileNotFoundError:
    print(f"❌ ERROR: '{output_filename}' not found.")
    print("Please ensure the file was downloaded correctly in the step above.")

# --- 2.4 Initial Data Inspection ---
print("\n--- First 5 Rows of the Raw Data ---")
display(df_raw.head())

print("\n--- Data Types and Missing Values ---")
df_raw.info()

### 3. Data Cleaning & Preprocessing

This is the most critical step of the analysis. Raw transactional data is notoriously "dirty." Before we can perform any feature engineering, we must create a clean, reliable analytical base. Our process will be systematic:

* **Handle Missing `CustomerID`s:** Transactions without a `CustomerID` are "guest checkouts." They are unusable for building customer profiles, so these rows must be removed.
* **Handle Duplicates:** We will check for and remove any exact duplicate rows to prevent double-counting.
* **Handle Data Types:** We will convert `CustomerID` from a float to a string (object) for proper grouping.
* **Handle Invalid Transactions:** We will filter out transactions with a `UnitPrice` of $0, as these do not represent a valid sale.
* **Separate Purchases vs. Cancellations:** We will create two distinct DataFrames:
    1.  `df_purchases`: All transactions with `Quantity > 0`. This will be our main dataset for analysis.
    2.  `df_cancellations`: All transactions where `InvoiceNo` starts with 'C'. We will keep this data to engineer our advanced `CancellationRate` feature later.

In [ ]:
# =============================================================================
# Block 3: Data Cleaning & Preprocessing
# =============================================================================

# We will work from the df_raw DataFrame loaded in Block 2

# --- 3.1 Handle Missing Values (CustomerID) ---
# We cannot segment customers we cannot identify.
initial_rows = df_raw.shape[0]
print(f"Initial row count: {initial_rows:,}")

# Check for missing CustomerIDs
missing_customers = df_raw['CustomerID'].isna().sum()
print(f"Rows with missing CustomerID: {missing_customers:,} ({missing_customers/initial_rows:.1%})")

# Drop rows where CustomerID is null
df_cleaned = df_raw.dropna(subset=['CustomerID'])
print(f"Row count after dropping missing CustomerIDs: {df_cleaned.shape[0]:,}")

# --- 3.2 Convert CustomerID to String ---
# CustomerID is a categorical identifier, not a numeric value.
# We convert it to string and remove the '.0' float suffix.
df_cleaned['CustomerID'] = df_cleaned['CustomerID'].astype(float).astype(int).astype(str)
print("\n✅ CustomerID converted to string type.")

# --- 3.3 Remove Duplicates ---
# Check for and drop any exact duplicate rows
initial_rows = df_cleaned.shape[0]
duplicates = df_cleaned.duplicated().sum()
print(f"\nFound {duplicates:,} duplicate rows.")

df_cleaned = df_cleaned.drop_duplicates()
print(f"Row count after dropping duplicates: {df_cleaned.shape[0]:,}")

# --- 3.4 Handle Invalid UnitPrice ---
# Filter out transactions with a UnitPrice of 0 or less
invalid_price = (df_cleaned['UnitPrice'] <= 0).sum()
print(f"\nFound {invalid_price:,} rows with UnitPrice <= 0.")

df_cleaned = df_cleaned[df_cleaned['UnitPrice'] > 0]
print(f"Row count after dropping invalid UnitPrice: {df_cleaned.shape[0]:,}")

# --- 3.5 Separate Purchases and Cancellations ---
# This is a key step for our advanced feature engineering.

# 3.5.1 Create df_cancellations
# These are all transactions where the InvoiceNo starts with 'C'
df_cancellations = df_cleaned[df_cleaned['InvoiceNo'].astype(str).str.startswith('C')]
print(f"\nFound {df_cancellations.shape[0]:,} cancellation entries.")

# 3.5.2 Create df_purchases
# These are all non-cancellation transactions with a Quantity > 0
df_purchases = df_cleaned[~df_cleaned['InvoiceNo'].astype(str).str.startswith('C')]
df_purchases = df_purchases[df_purchases['Quantity'] > 0]
print(f"Found {df_purchases.shape[0]:,} valid purchase entries.")

# --- 3.6 Final Check ---
print("\n✅ Data cleaning complete.")
print("\n--- Purchase Data Info ---")
df_purchases.info()

print("\n--- Cancellation Data Info ---")
df_cancellations.info()

print("\n--- First 5 Rows of Cleaned Purchase Data ---")
display(df_purchases.head())

### 4. Advanced Feature Engineering

This is the most valuable step in our analysis. We will transform the cleaned, transactional data (one row per *item*) into a customer-centric dataset (one row per *customer*).

This new table will contain our 8-feature **"Customer DNA" profile**, which goes far beyond a simple RFM (Recency, Frequency, Monetary) model.

Our 8 features are designed to answer three key questions about each customer:

1.  **Transactional Behavior (What & How):**
    * `Recency`: How recently did they buy?
    * `TotalOrders`: How many orders have they placed?
    * `TotalMonetary`: How much have they spent in total?
    * `AverageOrderValue`: Do they place big, high-value orders?
    * `AverageBasketSize`: Do they buy many items at once?
    * `ProductDiversity`: Do they buy a wide variety of products?

2.  **Loyalty (How long):**
    * `CustomerTenure`: How long have they been a customer? (A new customer with 5 orders is very different from a 3-year-old customer with 5 orders).

3.  **Profitability (How reliable):**
    * `CancellationRate`: What percentage of their orders do they cancel? (A high-spending but high-cancellation customer is not profitable).

We will calculate these features using our `df_purchases` and `df_cancellations` dataframes and then merge them into one final `df_final` table.

In [ ]:
# =============================================================================
# Block 4: Advanced Feature Engineering
# =============================================================================

# We will use df_purchases, df_cancellations, and df_cleaned from Block 3

# --- 4.1 Create TotalPrice Feature ---
# This is a prerequisite for all monetary calculations
df_purchases['TotalPrice'] = df_purchases['Quantity'] * df_purchases['UnitPrice']

# --- 4.2 Define Snapshot Date ---
# This is our reference point for "today" to calculate Recency and Tenure
snapshot_date = df_purchases['InvoiceDate'].max() + dt.timedelta(days=1)
print(f"Snapshot date defined as: {snapshot_date.date()}")

# --- 4.3 Aggregate Purchase & Loyalty Features ---
print("Calculating purchase and loyalty features...")
# We can get most of our features in a single aggregation from df_purchases
df_agg = df_purchases.groupby('CustomerID').agg(
    TotalMonetary=('TotalPrice', 'sum'),          # For TotalMonetary
    TotalOrders=('InvoiceNo', 'nunique'),       # For TotalOrders
    ProductDiversity=('StockCode', 'nunique'),    # For ProductDiversity
    TotalQuantity=('Quantity', 'sum'),          # To calculate AverageBasketSize
    LastPurchaseDate=('InvoiceDate', 'max'),    # To calculate Recency
    FirstPurchaseDate=('InvoiceDate', 'min')    # To calculate CustomerTenure
).reset_index()

# --- 4.4 Aggregate Profitability Features ---
print("Calculating profitability features...")
# Get total invoices from the *original* cleaned dataframe
df_total_invoices = df_cleaned.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
df_total_invoices.rename(columns={'InvoiceNo': 'TotalInvoices'}, inplace=True)

# Get cancelled invoices from the cancellations dataframe
df_cancelled_invoices = df_cancellations.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
df_cancelled_invoices.rename(columns={'InvoiceNo': 'CancelledInvoices'}, inplace=True)

# --- 4.5 Assemble Final "Customer DNA" Table ---
print("Assembling final feature table...")
# 1. Start with our aggregated purchase data
df_final = df_agg.copy()
# 2. Merge total invoice counts
df_final = pd.merge(df_final, df_total_invoices, on='CustomerID', how='left')
# 3. Merge cancelled invoice counts
df_final = pd.merge(df_final, df_cancelled_invoices, on='CustomerID', how='left')
# 4. FillNa for customers with 0 cancellations
df_final['CancelledInvoices'].fillna(0, inplace=True)

# --- 4.6 Calculate the 8 Final Features ---
print("Calculating all 8 'Customer DNA' features...")

# 1. Recency (days since last purchase)
df_final['Recency'] = (snapshot_date - df_final['LastPurchaseDate']).dt.days

# 2. CustomerTenure (days since first purchase)
df_final['CustomerTenure'] = (snapshot_date - df_final['FirstPurchaseDate']).dt.days

# 3. CancellationRate (profitability metric)
df_final['CancellationRate'] = df_final['CancelledInvoices'] / df_final['TotalInvoices']

# 4. AverageOrderValue (AOV)
df_final['AverageOrderValue'] = df_final['TotalMonetary'] / df_final['TotalOrders']

# 5. AverageBasketSize (items per order)
df_final['AverageBasketSize'] = df_final['TotalQuantity'] / df_final['TotalOrders']

# The other 3 features (TotalMonetary, TotalOrders, ProductDiversity) are already in the table.

# --- 4.7 Final Selection & Inspection ---
# Select our 8 features + CustomerID
feature_columns = [
    'Recency',
    'CustomerTenure',
    'TotalOrders',
    'TotalMonetary',
    'AverageOrderValue',
    'AverageBasketSize',
    'ProductDiversity',
    'CancellationRate'
]

df_final = df_final[['CustomerID'] + feature_columns]

# Handle any NaNs/Infs from division by zero (e.g., if TotalOrders was 0)
df_final.replace([np.inf, -np.inf], np.nan, inplace=True)
df_final.dropna(inplace=True)

print(f"\n✅ Feature engineering complete. Final customer profile shape: {df_final.shape}")
print(f"   (Found {df_final.shape[0]:,} customers with at least one valid purchase)")

print("\n--- First 5 Rows of 'Customer DNA' Table ---")
display(df_final.head())

print("\n--- Descriptive Statistics of Final Features ---")
display(df_final.describe().T)

### 5. Exploratory Data Analysis (EDA)

This section performs a comprehensive exploratory analysis of the 8-feature "Customer DNA" profile we engineered in the previous block.

#### 5.1 EDA Objectives

The primary objectives of this analysis are to:
1.  **Validate Feature Properties:** Quantify the statistical properties of each feature (central tendency, spread, and range).
2.  **Diagnose Data Quality:** Visually and statistically identify the presence of outliers and, critically, the **degree of skewness** in each feature's distribution.
3.  **Analyze Relationships:** Investigate the correlations and bivariate relationships between features to understand their interdependencies.
4.  **Assess Cluster Readiness:** Use advanced, non-linear dimensionality reduction (t-SNE) to project the 8-dimensional data into a visual "map," providing an initial assessment of natural, separable clusters.

#### 5.2 Diagnostic Expectation (Key Finding)

It is hypothesized that our raw, engineered features (especially those related to monetary value and time) will be **highly right-skewed** and contain **significant outliers**.

The following plots (histograms, box plots) are designed to *confirm* and *quantify* this hypothesis. Observing this severe skew is not an error; it is the **critical diagnostic finding** of our EDA. It provides the empirical justification for the **mandatory logarithmic transformation** and **standard scaling** in the next preprocessing step, which are required for our distance-based clustering algorithms to function correctly.

In [ ]:
# =============================================================================
# Block 5: Exploratory Data Analysis (EDA)
# =============================================================================

# We will analyze the df_final DataFrame created in Block 4
df_eda = df_final.drop('CustomerID', axis=1)

# --- 5.1 Descriptive & Skewness Analysis ---
print("--- 5.1 Descriptive & Skewness Analysis ---")
print("\nDescriptive Statistics:")
desc_stats = df_eda.describe().T
display(desc_stats)

print("\nSkewness of Features:")
# Skewness > 1 or < -1 indicates high skew
skew_df = pd.DataFrame(df_eda.skew(), columns=['Skewness'])
display(skew_df)

# --- 5.2 Distribution Analysis (Histograms) ---
print("\n--- 5.2 Generating Distribution (Histograms) ---")
# This figure is now *only* for histograms, making them larger.
plt.figure(figsize=(20, 12))
plt.suptitle("Histograms of Feature Distributions (Before Transformation)", fontsize=18)
for i, feature in enumerate(df_eda.columns):
    plt.subplot(3, 3, i+1)
    sns.histplot(df_eda[feature], bins=50, kde=False)
    plt.title(f'Distribution of {feature}', fontsize=12)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig("feature_histograms.png")
print("✅ feature_histograms.png saved.")
plt.show() # This displays the plot and creates a "gap"

# --- 5.3 Outlier Analysis (Box Plots) ---
print("\n--- 5.3 Generating Outlier Analysis (Box Plots) ---")
# A *new*, separate figure for box plots.
plt.figure(figsize=(20, 12))
plt.suptitle("Box Plots of Feature Distributions (Showing Outliers)", fontsize=18)
for i, feature in enumerate(df_eda.columns):
    plt.subplot(3, 3, i+1)
    sns.boxplot(y=df_eda[feature])
    plt.title(f'Box Plot of {feature}', fontsize=12)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig("feature_boxplots.png")
print("✅ feature_boxplots.png saved.")
plt.show() # Creates another "gap"

# --- 5.4 Relationship Analysis (Heatmap & Pairplot) ---
print("\n--- 5.4 Generating Relationship Analysis ---")

# A separate figure for the Correlation Heatmap
plt.figure(figsize=(12, 8))
corr_matrix = df_eda.corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of the 8 Features', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("feature_correlation_heatmap.png")
print("✅ feature_correlation_heatmap.png saved.")
plt.show() # Creates another "gap"

# A separate figure for the Targeted Pairplot
print("Generating targeted pairplot...")
pairplot_features = ['Recency', 'TotalMonetary', 'CustomerTenure', 'CancellationRate']
pairplot_fig = sns.pairplot(df_eda[pairplot_features], plot_kws={'alpha': 0.4, 's': 10})
pairplot_fig.fig.suptitle("Targeted Pairplot of Key Features", y=1.02, fontsize=16)
plt.savefig("targeted_pairplot.png")
print("✅ targeted_pairplot.png saved.")
plt.show() # Creates another "gap"

# --- 5.5 Cluster Readiness Check (t-SNE) ---
print("\n--- 5.5 Running t-SNE for Cluster Readiness Check ---")
# t-SNE is computationally expensive. We'll use a 10% sample.
df_sample = df_eda.sample(frac=0.10, random_state=42)

# We must scale the data *before* running t-SNE
scaler_eda = StandardScaler()
df_sample_scaled = scaler_eda.fit_transform(df_sample)

print(f"Running t-SNE on a sample of {len(df_sample)} customers... (This may take a minute)")
tsne_3d = TSNE(
    n_components=3,
    perplexity=30,
    random_state=42,
    n_iter=300,
    init='pca'
)
tsne_components_3d = tsne_3d.fit_transform(df_sample_scaled)

# Create a DataFrame for plotting
df_tsne_3d = pd.DataFrame(data = tsne_components_3d,
                          columns = ['TSNE1', 'TSNE2', 'TSNE3'])
print("t-SNE calculation complete.")

# --- 5.5.1 Plotting 3D t-SNE (Interactive) ---
print("Generating interactive 3D t-SNE plot...")
fig_3d_tsne = px.scatter_3d(
    df_tsne_3d,
    x='TSNE1',
    y='TSNE2',
    z='TSNE3',
    title='Interactive 3D t-SNE Projection (on 10% Sample)',
    opacity=0.6,
    width=900,
    height=700
)
fig_3d_tsne.update_traces(marker=dict(size=3))
fig_3d_tsne.write_html("tsne_3d_plot.html")
print("✅ Interactive 3D t-SNE plot saved as 'tsne_3d_plot.html'.")
# This plot will appear in its own interactive window.
fig_3d_tsne.show()

# --- 5.5.2 Plotting 2D t-SNE (Static) ---
print("\nGenerating 2D t-SNE plot...")
# A final, separate figure for the 2D t-SNE
plt.figure(figsize=(10, 7))
sns.scatterplot(data=df_tsne_3d, x='TSNE1', y='TSNE2', alpha=0.6, s=20)
plt.title('2D t-SNE Projection (Components 1 & 2)', fontsize=16)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid(True)
plt.savefig('tsne_2d_plot.png')
print("✅ 2D t-SNE plot saved as 'tsne_2d_plot.png'.")
plt.show()

print("\n--- ✅ EDA complete. All plots and stats saved. ---")

### 6. Final Preprocessing for Modeling

Our EDA in Block 5 definitively proved that our 8 features are highly skewed and have a wide range of scales. K-Means and DBSCAN are **distance-based algorithms**, meaning they are extremely sensitive to both of these issues.

This block performs the two **mandatory transformations** to make our data ready for these models:

1.  **Logarithmic Transformation:** We will apply a `np.log1p` transformation to our highly skewed features (e.g., `TotalMonetary`, `Recency`, `TotalOrders`). This "squashes" the extreme outliers and pulls the long tail in, converting the distribution to be more "normal."
2.  **Standard Scaling:** After logging, we will apply a `StandardScaler`. This final step rescales all 8 features so that they all have a **mean of 0 and a standard deviation of 1**.

After this block, all features will contribute equally to the model, and we will have a clean, robust `df_scaled` dataset ready for clustering.

In [ ]:
# =============================================================================
# Block 6: Final Preprocessing for Modeling
# =============================================================================

print(f"Original shape of data: {df_final.shape}")

# --- 6.1 Identify Features for Transformation ---
# We use the 'df_eda' (X) and 'df_final' (X + y) from earlier
# We will transform all features *except* our ratio, CancellationRate
# Note: AOV can be negative if returns > purchases in a period,
# but in our df_final (from df_purchases), min AOV should be > 0.
# We will use log1p (log(x+1)) to handle any values of 0.

# List of skewed features identified in EDA
skewed_features = [
    'Recency',
    'CustomerTenure',
    'TotalOrders',
    'TotalMonetary',
    'AverageOrderValue',
    'AverageBasketSize',
    'ProductDiversity'
]

# --- 6.2 Apply Logarithmic Transformation ---
# We create a new dataframe for our preprocessed data
df_processed = df_final.copy()

print(f"\nApplying log transform to {len(skewed_features)} features...")
for col in skewed_features:
    df_processed[col] = np.log1p(df_processed[col])

print("Log transform complete.")

# --- 6.3 Apply Standard Scaling ---
print("Applying standard scaling to all 8 features...")
# Set CustomerID as the index. It is an identifier, not a feature.
df_processed = df_processed.set_index('CustomerID')

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
# This creates a NumPy array, which is what our models expect
df_scaled = scaler.fit_transform(df_processed)

print("✅ Data successfully logged and scaled.")

# --- 6.4 Inspection of Processed Data ---
print(f"\nShape of final scaled data (NumPy array): {df_scaled.shape}")
print("This data is now ready for clustering.")

# We can also display the head of the *logged* data (before scaling)
print("\n--- Head of Log-Transformed Data (Before Scaling) ---")
display(df_processed.head())

### 7. Modeling Part 1: K-Means Clustering

Now that our data is fully preprocessed, we can begin the modeling phase. Our first approach is **K-Means Clustering**, a robust and popular algorithm.

#### 7.1 Algorithm & Objective

K-Means is a distance-based algorithm that partitions data into a pre-defined number of clusters (k). Its objective is to minimize the **Within-Cluster Sum of Squares (WCSS)**, also known as **inertia**. This is the sum of the squared distances between each data point and its assigned cluster's center (centroid). A lower WCSS means the clusters are more compact and well-defined.

#### 7.2 Finding the Optimal 'k' (The Elbow Method)

The main challenge with K-Means is that we must *tell it* how many clusters (k) to find. To find the optimal 'k', we will use the **Elbow Method**.

* **What it does:** We will run the K-Means algorithm for a range of 'k' values (e.g., 1 to 10) and plot the WCSS (inertia) for each 'k'.
* **How to read the plot:** As 'k' increases, the WCSS will always decrease. However, the "optimal" 'k' is found at the "elbow" — the point on the graph where the rate of decrease sharply flattens, resembling an arm. This point represents the best trade-off between minimizing WCSS and not having too many (over-fitted) clusters.

In [ ]:
# =============================================================================
# Block 7: Modeling Part 1 - K-Means Elbow Method
# =============================================================================

# We will use the df_scaled (NumPy array) from Block 6
# and the KMeans class imported in Block 1

print("--- 7.1 Running K-Means Elbow Method ---")

# We will store the WCSS (inertia) for each k
wcss = []
k_range = range(1, 11)  # We will test k from 1 to 10 clusters

for k in k_range:
    # Initialize K-Means
    # 'init'='k-means++' is a smart initialization technique
    # 'n_init'='auto' (or 10) runs the algorithm multiple times to find the best result
    # 'random_state'=42 ensures our results are reproducible
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)

    # Fit the model to our scaled data
    kmeans.fit(df_scaled)

    # Append the inertia (WCSS) to our list
    wcss.append(kmeans.inertia_)

print("WCSS values calculated for k=1 to 10.")

# --- 7.2 Plot the Elbow Method Graph ---
print("Generating Elbow Method plot...")
plt.figure(figsize=(10, 6))
plt.plot(k_range, wcss, marker='o', linestyle='--')
plt.title('K-Means Elbow Method', fontsize=16)
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS (Inertia)')
plt.xticks(k_range)
plt.grid(True)
plt.savefig('kmeans_elbow_plot.png')
print("✅ kmeans_elbow_plot.png saved.")
plt.show()

### 7. Modeling Part 2: K-Means Execution & Cluster Assignment

Based on the Elbow Method plot from the previous step, we can identify the optimal number of clusters (k). The "elbow" appears to be at **k=4** (or k=3, depending on the plot). This indicates that 4 clusters is the best balance between model complexity and explanatory power (WCSS).

We will now re-run the K-Means algorithm with our chosen `k=4` and assign the resulting cluster labels back to our customer data.

* **What it does:**
    1.  Runs the `KMeans` algorithm on our `df_scaled` data with `n_clusters=4`.
    2.  Assigns the resulting cluster label (0, 1, 2, or 3) to each customer.
    3.  Adds these new labels back to both our `df_final` (original values) and `df_processed` (log-transformed values) DataFrames.
* **Why we do this:** This is the core segmentation step. By adding the cluster labels back to our human-readable DataFrames, we can now move on to the most important part of the project: **Cluster Profiling**. This allows us to analyze the *characteristics* of each segment.

In [ ]:
# =============================================================================
# Block 8: Modeling Part 2 - K-Means Execution
# =============================================================================

# We will use the df_scaled (NumPy array) from Block 6
# and df_final / df_processed from Blocks 4 & 6

# --- 8.1 Set Optimal k ---
# *** ACTION REQUIRED: ***
# Based on your elbow plot from Block 7, set the optimal 'k' here.
# We will proceed with k=4 as a common example.
OPTIMAL_K = 4

print(f"--- 8.1 Running K-Means with optimal k={OPTIMAL_K} ---")

# Initialize and run the final K-Means model
kmeans = KMeans(n_clusters=OPTIMAL_K, init='k-means++', n_init=10, random_state=42)
kmeans.fit(df_scaled)

# Get the cluster labels for each customer
cluster_labels = kmeans.labels_

# --- 8.2 Assign Cluster Labels Back to DataFrames ---
# We add the labels to both our original and processed data for analysis

# 1. Add to the log-transformed data (df_processed)
df_processed['Cluster'] = cluster_labels

# 2. Add to the original-value data (df_final)
# We must be careful to match the index, as df_final has CustomerID as a column
# df_processed has CustomerID as an index.
df_final_clustered = df_final.set_index('CustomerID')
df_final_clustered['Cluster'] = cluster_labels
df_final_clustered = df_final_clustered.reset_index()

print("✅ Cluster labels assigned back to DataFrames.")

# --- 8.3 Inspect the Results ---
print(f"\n--- Cluster Distribution (Size of Each Segment) ---")
# Display the number of customers in each cluster
print(df_final_clustered['Cluster'].value_counts().sort_index())

print("\n--- Head of Final Clustered Data (Original Values) ---")
display(df_final_clustered.head())

print("\n--- Head of Processed Clustered Data (Logged Values) ---")
display(df_processed.head())

### 8. Cluster Profiling & Interpretation

This is the most important part of the entire analysis. We have successfully segmented our customers into 4 (or your `OPTIMAL_K`) distinct groups. Now, we must figure out *who these groups are*.

#### 8.1 Objective

Our goal is to create a "persona" for each cluster. We will do this by grouping all customers by their assigned cluster and calculating the average value for each of the 8 features.

* **What it does:** We will use our `df_final_clustered` DataFrame (which contains the **original, human-readable values**) to calculate the mean of `Recency`, `CustomerTenure`, `TotalMonetary`, `CancellationRate`, etc., for each cluster.
* **Why we do this:** This allows us to compare the clusters side-by-side. For example, we might find:
    * **Cluster 0:** Has LOW Recency, HIGH TotalMonetary, HIGH TotalOrders... (These are our "Champions").
    * **Cluster 1:** Has HIGH Recency, LOW TotalOrders, LOW Tenure... (These are our "Lost Customers").
    * **Cluster 2:** Has LOW Recency, LOW Tenure, HIGH CancellationRate... (These are our "At-Risk New Customers").

This profile is what we deliver to the business to build marketing strategies.

In [ ]:
# =============================================================================
# Block 9: Cluster Profiling - Statistical Analysis
# =============================================================================

# We will use the df_final_clustered DataFrame from Block 8,
# as it contains the *original, unscaled* values, which are human-readable.

print("--- 9.1 Calculating Cluster Profiles (Averages) ---")

# Group by the 'Cluster' label and calculate the mean for all 8 features
# We select all feature columns + 'Cluster'
profile_features = feature_columns + ['Cluster']
df_profile = df_final_clustered[profile_features].groupby('Cluster').mean()

# We can also add the 'size' of each cluster to this profile
df_profile['ClusterSize'] = df_final_clustered['Cluster'].value_counts().sort_index()

# Transpose (.T) the DataFrame for a much cleaner, vertical "persona" view
df_profile_t = df_profile.T

print("✅ Cluster profiling complete.")

# --- 9.2 Display the Cluster Profiles ---
print("\n--- Cluster Profile Table (Averages per Cluster) ---")
display(df_profile_t)

# --- 9.3 Plot Cluster Sizes ---
print("\n--- 9.3 Generating Cluster Size Plot ---")
plt.figure(figsize=(8, 5))
sns.countplot(data=df_final_clustered, x='Cluster', palette='viridis')
plt.title('Customer Segment Size Distribution', fontsize=16)
plt.xlabel('Cluster ID')
plt.ylabel('Number of Customers')
plt.savefig('cluster_size_barchart.png')
print("✅ cluster_size_barchart.png saved.")
plt.show()

### 9. Cluster Profiling & Visualization

This is the most critical stage of the analysis: **interpretation**. We have successfully segmented our customers into 4 (or your `OPTIMAL_K`) distinct groups. Now, we must figure out *who these groups are*.

#### 9.1 Statistical Profiling

Our first step is to create a statistical "persona" for each cluster.

* **What it does:** We will group all customers by their assigned cluster and calculate the average value for each of our 8 features. We will use the `df_final_clustered` DataFrame, which contains the **original, human-readable values** (not the scaled ones).
* **Why we do this:** This allows us to compare the clusters side-by-side. For example, we might find:
    * **Cluster 0:** Has LOW `Recency`, HIGH `TotalMonetary`, HIGH `TotalOrders`... (These are our "Champions").
    * **Cluster 1:** Has HIGH `Recency`, LOW `TotalOrders`, LOW `CustomerTenure`... (These are our "Lost Customers").

This profile is what we deliver to the business to build marketing strategies.

#### 9.2 Visual Profiling (Snake Plot)

A table of averages can be hard to read. A **"Snake Plot"** is an advanced visualization that plots the *scaled* average value of each feature for all clusters on a single line graph.

* **What it does:** It shows the "profile" of each cluster as a "snake."
* **Why we do this:** It makes it incredibly easy to see the differences. We can instantly spot which cluster is "high" on `TotalMonetary` but "low" on `Recency`. We will use our `df_processed` (log-transformed and scaled) data for this plot, as all features are on the same scale (mean of 0, std of 1).

In [ ]:
# =============================================================================
# Block 9: Cluster Profiling & Visualization
# =============================================================================

# We will use df_final_clustered (original values) and df_processed (scaled values)

# --- 9.1 Statistical Profiling ---
print("--- 9.1 Calculating Cluster Profiles (Averages) ---")

# We use the original, unscaled values for human-readable interpretation
profile_features = feature_columns + ['Cluster']
df_profile = df_final_clustered[profile_features].groupby('Cluster').mean()

# Add the 'ClusterSize' to our profile table
df_profile['ClusterSize'] = df_final_clustered['Cluster'].value_counts().sort_index()

# Transpose (.T) the DataFrame for a much cleaner, vertical "persona" view
df_profile_t = df_profile.T

print("✅ Cluster profiling complete.")

# --- 9.2 Display the Cluster Profiles ---
print("\n--- Cluster Profile Table (Averages per Cluster) ---")
# This table is the main result for defining our personas
display(df_profile_t)

# --- 9.3 Plot Cluster Sizes ---
print("\n--- 9.3 Generating Cluster Size Plot ---")
plt.figure(figsize=(8, 5))
# We use the 'df_final_clustered' which has the cluster assignments
sns.countplot(data=df_final_clustered, x='Cluster', palette='viridis')
plt.title('Customer Segment Size Distribution', fontsize=16)
plt.xlabel('Cluster ID')
plt.ylabel('Number of Customers')
plt.savefig('cluster_size_barchart.png')
print("✅ cluster_size_barchart.png saved.")
plt.show()

# --- 9.4 Visual Profiling (Snake Plot) ---
print("\n--- 9.4 Generating Snake Plot (Scaled Averages) ---")

# We use the *processed* (logged and scaled) dataframe for this
# This ensures all features are on the same scale (z-score)
df_profile_scaled = df_processed.groupby('Cluster').mean()

# Melt the dataframe from wide to long format for plotting with seaborn
df_snake_plot = pd.melt(df_profile_scaled.reset_index(),
                        id_vars='Cluster',
                        value_vars=feature_columns,
                        var_name='Feature',
                        value_name='ScaledValue')

plt.figure(figsize=(14, 7))
sns.lineplot(data=df_snake_plot, x='Feature', y='ScaledValue', hue='Cluster',
             palette='viridis', marker='o', sort=False)
plt.title('Snake Plot: Comparing Scaled Cluster Averages', fontsize=16)
plt.xlabel('Feature')
plt.ylabel('Scaled Average Value (Z-Score)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Cluster')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.axhline(0, color='black', linestyle='--', linewidth=1) # Add a 'mean' line
plt.tight_layout()
plt.savefig('cluster_snake_plot.png')
print("✅ cluster_snake_plot.png saved.")
plt.show()

### 10. K-Means: Cluster Interpretation & Personas

This is the final, most important step of our K-Means analysis. Using the profile table and snake plot from Block 9, we will now define *who* these customers are by giving each cluster a descriptive "persona."

* **Methodology:** We analyze the 8-feature averages for each cluster, comparing them to the other clusters to find their unique characteristics.

#### **Cluster Persona Analysis:**

* **Cluster 0: "High-Value Churn Risk" (Size: 1,185)**
    * **Profile:** This group has a high Average Order Value ($715) and Basket Size (451 items), meaning they place large, valuable orders.
    * **Problem:** They have a high Recency (126 days) and a very high Cancellation Rate (18%).
    * **Interpretation:** These are valuable customers who are **fading away**. They haven't shopped in a long time and are unreliable, costing the business money with cancellations.
    * **Strategy:** Target with personalized win-back campaigns, but be cautious due to their high cancellation habit.

* **Cluster 1: "Lost Customers" (Size: 1,151)**
    * **Profile:** This is the worst-performing group across almost every metric: *highest* Recency (174 days), *lowest* Orders, *lowest* Monetary value, and *lowest* Diversity.
    * **Problem:** They have churned.
    * **Interpretation:** These are low-value customers who stopped shopping almost 6 months ago.
    * **Strategy:** Do not invest heavily. A single, automated "we miss you" email with a deep discount is the maximum effort.

* **Cluster 2: "Champions" (Size: 1,212)**
    * **Profile:** This is our **best and most valuable segment.** They are the top performers in every key category: *lowest* Recency (20 days), *highest* Tenure, *highest* Total Orders, *highest* Total Monetary value, and *highest* Product Diversity.
    * **Problem:** Their only minor flaw is a medium-high Cancellation Rate (14%).
    * **Interpretation:** These are our loyal, engaged, high-spending VIPs.
    * **Strategy:** **Retain & Reward.** Offer loyalty perks, early access, and exclusive content. Do *not* use discounts. Investigate their cancellations to improve their experience.

* **Cluster 3: "New & Promising" (Size: 790)**
    * **Profile:** This group has the *lowest* Tenure (49 days), meaning they are brand new. They have good Recency (32 days) and the *lowest* Cancellation Rate (4%).
    * **Problem:** Their spending and order counts are still low (as expected).
    * **Interpretation:** These are our newest customers, and they are behaving well (low cancellations). They represent the future growth of the business.
    * **Strategy:** **Nurture & Grow.** Onboard them with a welcome series. Encourage their second and third purchase with targeted follow-ups.

### 11. Modeling Part 3: DBSCAN Clustering

Now that we have a complete analysis from K-Means, we will run a second, more advanced model: **DBSCAN (Density-Based Spatial Clustering of Applications with Noise)**.

#### 11.1 Algorithm & Objective

DBSCAN is a powerful density-based model. Unlike K-Means, it does *not* require us to specify the number of clusters. Instead, it finds clusters based on how "dense" the data is.

Its great advantage is that it can identify **noise points (outliers)**. K-Means forces every customer into a cluster, even if they don't fit. DBSCAN will label these outliers as a separate group (labeled "-1"), which is an extremely valuable business segment (e.g., "Anomalous Customers," "One-Time Bulk Buyers," or "Fraudulent Accounts").

* **`eps` (Epsilon):** The maximum distance between two points for one to be considered in the "neighborhood" of the other. This is the most important parameter to tune.
* **`min_samples`:** The minimum number of points required to form a dense "core" region.

In [ ]:
# =============================================================================
# Block 10: Modeling Part 3 - DBSCAN Clustering
# =============================================================================

# We will use the *same* df_scaled (NumPy array) from Block 6
# This allows for a fair, direct comparison with our K-Means results

print("--- 10.1 Running DBSCAN ---")

# --- 10.1.1 Tuning DBSCAN ---
# Finding the right 'eps' is key. A common method is to find the 'knee'
# in a k-nearest neighbors plot. For 8-dimensional data, this is complex.
# A good starting point is to test values based on the data's properties.
# Given our 8 features (all scaled to mean=0, std=1), a distance of
# 2.5 - 3.5 is a reasonable starting point to test.
# 'min_samples' is often set to 2 * number_of_dimensions
MIN_SAMPLES = 2 * df_scaled.shape[1]  # 2 * 8 = 16
EPS = 3.0  # This is a hyperparameter to tune. We'll start with 3.0

dbscan = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES)
dbscan_labels = dbscan.fit_predict(df_scaled)

# --- 10.2 Analyze DBSCAN Results ---
# The label '-1' is given to all "noise" points (outliers)
n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)
n_clustered = len(dbscan_labels) - n_noise

print(f"✅ DBSCAN clustering complete.")
print(f"   Parameters: eps={EPS}, min_samples={MIN_SAMPLES}")
print(f"\n--- DBSCAN Results ---")
print(f"Total Clusters Found: {n_clusters}")
print(f"Clustered Customers:  {n_clustered} ({n_clustered/len(dbscan_labels):.1%})")
print(f"Outlier Customers (-1): {n_noise} ({n_noise/len(dbscan_labels):.1%})")

# --- 10.3 Add DBSCAN Labels to DataFrames ---
# We can now add these new labels to compare with our K-Means results
df_final_clustered['DBSCAN_Cluster'] = dbscan_labels
df_processed['DBSCAN_Cluster'] = dbscan_labels

print("\n--- DBSCAN Cluster Size Distribution ---")
print(df_final_clustered['DBSCAN_Cluster'].value_counts())

print("\n--- Head of Final Data with Both Cluster Labels ---")
display(df_final_clustered.head())

### 11. Modeling Part 4: DBSCAN Hyperparameter Tuning

The DBSCAN run in Block 10 failed, identifying 99.7% of all customers as a single cluster. This is a classic symptom of an `eps` (Epsilon) value that is far too large.

To find the optimal `eps`, we cannot "guess." We must use a systematic method. The standard approach is to calculate the distance to the k-th nearest neighbor for every point, where 'k' is our `min_samples` (which we set to 16).

* **What it does:** We will calculate the distance for every customer to its 16th nearest neighbor, sort these distances, and plot them.
* **How to read the plot:** This plot will show a "knee" or "elbow." The y-value of this "knee" is the optimal `eps`. It represents the point where the distance between points suddenly jumps, which is the exact definition of a cluster border.

In [ ]:
# =============================================================================
# Block 11: DBSCAN Hyperparameter Tuning (Epsilon Knee Plot)
# =============================================================================

# We will use the same df_scaled data
from sklearn.neighbors import NearestNeighbors

# --- 11.1 Calculate K-NN Distances ---
# We use min_samples (16) as our 'k' (n_neighbors)
# MIN_SAMPLES was set to 16 in the previous block
k_neighbors = MIN_SAMPLES

print(f"Calculating distances for {k_neighbors}-Nearest Neighbors...")

# Initialize and fit the model
neighbors = NearestNeighbors(n_neighbors=k_neighbors)
neighbors.fit(df_scaled)

# Find the distances to the k-th neighbor for all points
distances, indices = neighbors.kneighbors(df_scaled)

# Get *only* the distance to the k-th (16th) neighbor (the last one)
k_distances = distances[:, k_neighbors-1]

# Sort the distances
sorted_distances = np.sort(k_distances)

print("Distances calculated and sorted.")

# --- 11.2 Plot the Epsilon Knee ---
print("Generating Epsilon 'Knee' Plot...")
plt.figure(figsize=(10, 6))
plt.plot(sorted_distances)
plt.title('Epsilon (eps) "Knee" Plot', fontsize=16)
plt.xlabel('Customers (Sorted by Distance)')
plt.ylabel(f'Distance to {k_neighbors}-th Neighbor (eps)')
plt.grid(True)
plt.savefig('dbscan_knee_plot.png')
print("✅ dbscan_knee_plot.png saved.")
plt.show()

# --- 11.3 How to Read This Plot ---
print("\n--- How to Read This Plot ---")
print("Look for the 'knee' in the plot (the point of maximum curvature).")
print("The Y-axis value at that 'knee' is your new, optimal 'eps' value.")
print("It will likely be much smaller than 3.0 (e.g., somewhere between 1.5 and 2.5).")

### 11. Modeling Part 5: Re-running DBSCAN with Optimal `eps`

The "Epsilon Knee Plot" from the previous block gives us a clear, data-driven value for our `eps` hyperparameter.

* **Finding:** After re-examining the plot, the "knee" (the point of maximum curvature where the slope first changes dramatically) is located at a Y-axis value of approximately **1.8**.
* **Interpretation:** This means the optimal distance to define a "neighborhood" is 1.8. Our previous value of 3.0 (and 2.4) was too large, causing the model to see most points as one giant cluster. This new, smaller `eps` should result in a much more granular and accurate segmentation.
* **Action:** We will now re-run DBSCAN using our new, correct value of `eps = 1.8` and the same `min_samples = 16`.

In [ ]:
# =============================================================================
# Block 12: Re-running DBSCAN with Optimal Epsilon (1.8)
# =============================================================================

# We will use the *same* df_scaled (NumPy array) from Block 6

# --- 12.1 Set Optimal Parameters ---
# From our corrected knee plot analysis
OPTIMAL_EPS = 1.8
MIN_SAMPLES = 16  # (This remains 2 * number_of_features)

print(f"--- 12.1 Re-running DBSCAN with optimal parameters ---")
print(f"   Parameters: eps={OPTIMAL_EPS}, min_samples={MIN_SAMPLES}")

# --- 12.2 Run the final DBSCAN model ---
dbscan_final = DBSCAN(eps=OPTIMAL_EPS, min_samples=MIN_SAMPLES)
dbscan_labels_final = dbscan_final.fit_predict(df_scaled)

# --- 12.3 Analyze New DBSCAN Results ---
n_clusters_final = len(set(dbscan_labels_final)) - (1 if -1 in dbscan_labels_final else 0)
n_noise_final = list(dbscan_labels_final).count(-1)
n_clustered_final = len(dbscan_labels_final) - n_noise_final

print(f"\n✅ DBSCAN (final run) complete.")
print(f"\n--- Final DBSCAN Results ---")
print(f"Total Clusters Found: {n_clusters_final}")
print(f"Clustered Customers:  {n_clustered_final} ({n_clustered_final/len(dbscan_labels_final):.1%})")
print(f"Outlier Customers (-1): {n_noise_final} ({n_noise_final/len(dbscan_labels_final):.1%})")

# --- 12.4 Add Final DBSCAN Labels to DataFrames ---
df_final_clustered['DBSCAN_Cluster_Final'] = dbscan_labels_final
df_processed['DBSCAN_Cluster_Final'] = dbscan_labels_final

print("\n--- Final DBSCAN Cluster Size Distribution ---")
# This output will show us the new, better cluster sizes
print(df_final_clustered['DBSCAN_Cluster_Final'].value_counts())

### 12. Model Validation: Feature Importance for K-Means

Our K-Means model (Block 8) found 4 clusters, but our DBSCAN model (Block 12) found only 1. This is a critical conflict. We must now **validate our K-Means results**.

Are the 4 clusters meaningful, or are they an artificial result of the algorithm?

To find out, we will use the **Permutation Importance** test.

#### 12.1 Methodology (The "Shuffle Test")

1.  **Create a "Target":** We will use the 4 cluster labels from our K-Means model (0, 1, 2, 3) as our "target variable."
2.  **Train a "Surrogate" Model:** We will train a new, supervised classifier (a Random Forest) to *predict* which cluster a customer belongs to, based on our 8 scaled features.
3.  **Run the "Shuffle Test":** We will then test this Random Forest's accuracy. We will shuffle *one feature* at a time (e.g., shuffle all `Recency` values) and see how much the model's accuracy drops.

If shuffling a feature (like `Recency`) causes the model's accuracy to collapse, it proves that `Recency` was **critically important** for building those clusters. If shuffling a feature does nothing, that feature was just noise.

In [ ]:
# =============================================================================
# Block 13: Feature Importance Analysis (Validating K-Means)
# =============================================================================

# We need a classifier and the importance function
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

print("--- 13.1 Validating K-Means with a Surrogate Model ---")

# --- 1. Define our X and y ---
X = df_scaled  # Our 8 scaled features
# 'cluster_labels' is the NumPy array from our K-Means run in Block 8
y = cluster_labels

# --- 2. Split into Training and Testing sets ---
# This is crucial for a valid permutation importance test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# --- 3. Train the Random Forest "Surrogate" Model ---
print("Training Random Forest classifier to predict K-Means clusters...")
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Check how well it learned the clusters
accuracy = rf_classifier.score(X_test, y_test)
print(f"Surrogate Model Accuracy: {accuracy:.2%}")
print("(A high accuracy > 95% means the clusters are well-defined by the features)")

# --- 4. Run the "Shuffle Test" (Permutation Importance) ---
print("\nRunning Permutation Importance (Shuffle Test)...")
# We run this on the *test* set to get an unbiased view
result = permutation_importance(
    rf_classifier, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
)

# --- 5. Organize and Plot the Results ---
#
# *** THIS IS THE FIX: ***
# We use our 'feature_columns' variable, which we created in Block 4.
# It contains the correct, clean list of our 8 feature names.
#
feature_names = feature_columns

# Create a clean DataFrame of the results
importance_df = pd.DataFrame(
    result.importances_mean,
    index=feature_names,
    columns=['Importance']
).sort_values(by='Importance', ascending=True)

print("\n--- Feature Importance for K-Means Clusters ---")
print(importance_df)

# --- 6. Plot the results ---
print("\nGenerating Feature Importance plot...")
plt.figure(figsize=(10, 7))
importance_df.plot(kind='barh', legend=False)
plt.title('K-Means Feature Importance (Permutation Test)', fontsize=16)
plt.xlabel('Importance (Drop in Model Accuracy)')
plt.ylabel('Feature')
plt.grid(True, axis='x', linestyle='--')
plt.tight_layout()
plt.savefig('kmeans_feature_importance.png')
print("✅ kmeans_feature_importance.png saved.")
plt.show()

### 13. Analysis of Feature Importance (The "Shuffle Test")

This block is the final validation of our entire K-Means model. The results are extremely positive and provide two critical insights.

#### Insight 1: Our K-Means Clusters are "Real" and Meaningful

* **`Surrogate Model Accuracy: 94.62%`**
* **Interpretation:** This is an outstanding result. It means our "surrogate" Random Forest model was able to predict which cluster a customer belonged to with ~95% accuracy, using only our 8 features.
* **Why it Matters:** This proves that our 4 K-Means clusters are **not** an artificial illusion. They are well-defined, mathematically separable groups that are strongly defined by our features. This gives us high confidence in our 4 personas.

#### Insight 2: `CustomerTenure` is the Most Important Feature

* **`Importance Rankings:`**
    1.  **`CustomerTenure` (0.17)**
    2.  `Recency` (0.11)
    3.  `TotalMonetary` (0.11)
    4.  `TotalOrders` (0.09)
* **Interpretation:** The "shuffle test" shows that `CustomerTenure` (our advanced, non-RFM feature) was the **single most important factor** in defining the clusters. This is a huge win for our 8-feature model. `Recency` and `TotalMonetary` are also (predictably) very important.
* **Why it Matters:** This proves our 8-feature model is **not overfitting**. We didn't just add noise; we added meaningful "signals." It also validates our K-Means personas. For example, our "Champions" vs. "New & Promising" clusters were separated *primarily* by their `CustomerTenure`.

This test also explains why DBSCAN "failed." DBSCAN looks for *density*, but our clusters are defined by a complex interplay of 8 different features (led by `Tenure`), not just density. This confirms that **K-Means was the correct model for this problem.**

### 14. Project Conclusion & Final Recommendations

This notebook successfully performed an end-to-end customer segmentation analysis, transforming raw, transactional data into four distinct, actionable customer personas.

#### Summary of Findings:

1.  **Advanced Model Justified:** Our 8-feature "Customer DNA" profile (which included `CustomerTenure`, `CancellationRate`, etc.) was validated by a Permutation Importance test. The test proved our clusters were meaningful (94.6% accuracy) and that our new features (especially `CustomerTenure`) were the most important drivers in segmenting customers.

2.  **K-Means Was the Correct Model:** K-Means successfully identified 4 center-based clusters. DBSCAN (a density-based model) correctly identified one single, dense "blob," proving that our customers are not separated by density but by their complex behavioral profiles.

3.  **Four Actionable Segments Identified:** We have successfully profiled and named our 4 K-Means clusters:
    * **Cluster 2: "Champions" (1,212 customers):** Our loyal, high-spending VIPs. (Strategy: **Retain & Reward**)
    * **Cluster 3: "New & Promising" (790 customers):** Our new, well-behaved customers. (Strategy: **Nurture & Grow**)
    * **Cluster 0: "High-Value Churn Risk" (1,185 customers):** High-value but high-cancellation customers who are fading away. (Strategy: **Win-Back & Investigate**)
    * **Cluster 1: "Lost Customers" (1,151 customers):** Low-value, churned customers. (Strategy: **Do Not Invest**)

#### Future Work & Recommendations:

* **Deeper Dive into "Churn Risk":** Cluster 0 is our biggest risk and opportunity. A follow-up analysis should investigate *why* their `CancellationRate` is so high.
* **Predictive Modeling:** This project was descriptive ("What do our segments look like?"). The next step is a predictive project: "Can we build a model to predict which new customers (Cluster 3) will become 'Champions' (Cluster 2)?"
* **Deliver to Marketing:** These 4 segments are ready to be used. They can now be exported and used to build 4 distinct, personalized marketing campaigns.

In [ ]:
# =============================================================================
# Block 14: Export Final Segments
# =============================================================================

# We use the 'df_final_clustered' DataFrame from Block 8
# It contains the CustomerID and the final K-Means 'Cluster' label

print("--- 14.1 Preparing Final Export ---")

# We can also add the 'Persona' name based on our analysis from Block 10
# This makes the file even more valuable to the business.
persona_map = {
    0: "High-Value Churn Risk",
    1: "Lost Customers",
    2: "Champions",
    3: "New & Promising"
}

# Add a new 'Persona' column
df_final_clustered['Persona'] = df_final_clustered['Cluster'].map(persona_map)

# Select the columns for the final export
df_export = df_final_clustered[['CustomerID', 'Cluster', 'Persona']]

# --- 14.2 Save to CSV ---
output_filename = "customer_segments.csv"
try:
    df_export.to_csv(output_filename, index=False)
    print(f"\n✅ Successfully exported {len(df_export)} customers to '{output_filename}'.")
except Exception as e:
    print(f"❌ Error saving file: {e}")

# --- 14.3 Display Final Exportable Data ---
print("\n--- Head of Final Export File ---")
display(df_export.head())

print("\n\n--- PROJECT COMPLETE ---")